In [5]:
import cv2
print("cv2 버전: ", cv2.__version__)

#카메라 캡처 객체, 0=내장카메라
capture = cv2.VideoCapture(0)

#캡처 프레임 사이즈 조절
capture.set(cv2.CAP_PROP_FRAME_WIDTH, 320)
capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 240)

while True:
    #한 프레임씩 읽기
    ret, frame = capture.read()
    if ret == True:
        print("read success!")
        
    #이미지 뒤집기, 1=좌우반전
    frame_fliped = cv2.flip(frame, 1)
    
    #읽어들인 프레임을 윈도우 창에 출력
    cv2.imshow("VideoFrame", frame_fliped)
    
    #1ms 동안 사용자가 키를 누르기를 기다림
    if cv2.waitKey(1) > 0:
        break
        
#카메라 객체 반환
capture.release()

#화면에 나타난 윈도우를 종료
cv2.destroyAllWindows()

cv2 버전:  4.5.5
read success!
read success!
read success!
read success!
read success!
read success!
read success!
read success!
read success!
read success!
read success!
read success!
read success!
read success!
read success!
read success!
read success!
read success!
read success!
read success!
read success!
read success!
read success!
read success!
read success!
read success!
read success!
read success!
read success!
read success!
read success!
read success!
read success!
read success!
read success!
read success!
read success!
read success!
read success!
read success!
read success!
read success!
read success!
read success!
read success!
read success!
read success!
read success!
read success!
read success!
read success!
read success!
read success!
read success!
read success!
read success!
read success!
read success!
read success!
read success!
read success!
read success!
read success!
read success!
read success!
read success!
read success!
read success!
read success!
read success!
read 

In [6]:
import tensorflow.keras
from PIL import Image, ImageOps
import numpy as np

# Disable scientific notation for clarity
np.set_printoptions(suppress=True)

# Load the model
model = tensorflow.keras.models.load_model('D:/22KJH/project/Python/6google_teachable_machine/keras_model.h5')

# Create the array of the right shape to feed into the keras model
# The 'length' or number of images you can put into the array is
# determined by the first position in the shape tuple, in this case 1.
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
# Replace this with the path to your image
image = Image.open('D:/22KJH/project/Python/6google_teachable_machine/case2.jpg')
#resize the image to a 224x224 with the same strategy as in TM2:
#resizing the image to be at least 224x224 and then cropping from the center
size = (224, 224)
image = ImageOps.fit(image, size, Image.ANTIALIAS)

#turn the image into a numpy array
image_array = np.asarray(image)
# Normalize the image
normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1
# Load the image into the array
data[0] = normalized_image_array

# run the inference
prediction = model.predict(data)
print(prediction)


[[0.00001242 0.9999876 ]]


In [8]:
import beepy
import kakao_utils

#컴퓨터에 내장된 소리를 출력
def beepsound():
    beepy.beep(sound=6)
    
    
#카카오톡 메세지로 영상 링크 전송
def send_video_link():
    KAKAO_TOKEN_FILENAME = "D:/22KJH/project/Python/6google_teachable_machine/kakao_token.json"
    KAKAO_APP_KEY = "5667bb7ce28bd6798e5f51bd12b0c0fc"
    tokens = kakao_utils.update_tokens(KAKAO_APP_KEY, KAKAO_TOKEN_FILENAME)
    
    #텍스트 메세지 보내기
    template = {
        "object_type": "text",
        "text" : "30초 이상 움직임이 감지되지 않았습니다. 졸지마세요!",
        "link" : {
            'web_url': "https://www.youtube.com/watch?v=QE1FiLCvcj0",
            "mobile_web_url": "https://www.youtube.com/watch?v=QE1FiLCvcj0"
        },
        "button_title": "잠 깨러 가기"
    }
    
    #카카오톡 메세지 전송
    res = kakao_utils.send_message(KAKAO_TOKEN_FILENAME, template)
    if res.json().get('result_code') == 0:
        print('텍스트 메세지를 성공적으로 보냈습니다.')
    else:
        print('텍스트 메세지를 보내지 못했습니다. 오류 메세지: ', res.json())
#beepsound()
#send_video_link()

In [9]:
import cv2
import tensorflow.keras
import numpy as np

#이미지 전처리
def preprocessing(frame):
    #크기 조정
    size = (224, 224)
    frame_resized = cv2.resize(frame, size, interpolation=cv2.INTER_AREA)
    
    # 이미지 정규화
    frame_normalized = (frame_resized.astype(np.float32) / 127.0) - 1
    
    #이미지 차원 재조정
    frame_reshaped = frame_normalized.reshape((1, 224, 224, 3))
    
    return frame_reshaped
                                              
#학습된 모델 불러오기
model_filename = 'D:/22KJH/project/Python/6google_teachable_machine/keras_model.h5'
model = tensorflow.keras.models.load_model(model_filename)

#카메라 캡처 객체, 0=내장카메라
capture = cv2.VideoCapture(0)

#캡처 프레임 사이즈 조절
capture.set(cv2.CAP_PROP_FRAME_WIDTH, 320)
capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 240)

sleep_cnt = 1 #30초간 '졸림'상태(case2) 확인하기 위한 변수
while True:
    #한 프레임씩 읽기
    ret, frame = capture.read()
    if ret == True:
        print("read success!")
        
    #이미지 뒤집기, 1=좌우반전
    frame_fliped = cv2.flip(frame, 1)
    
    #읽어들인 프레임을 윈도우 창에 출력
    cv2.imshow("VideoFrame", frame_fliped)
    
    #1초마다 검사, videoframe 창으로 아무키나 누르게 되면 종료
    if cv2.waitKey(200) > 0:
        break
        
    #데이터 전처리
    preprocessed = preprocessing(frame_fliped)
    
    #예측
    prediction = model.predict(preprocessed)
    
    if prediction[0,0] < prediction[0, 1]:
        print('졸림 상태')
        sleep_cnt += 1
    
        #졸림 상태가 30초간 지속되면 소리&카톡 보내기
        if sleep_cnt % 10 == 0:
            sleep_cnt = 1
            print('30초 이상 움직임이 감지되지 않았습니다. 졸지마세요!')
            beepsound()
            send_video_link()
            break
    else:
        print('깨어있는 상태')
        sleep_cnt = 1
            
#카메라 객체 반환
capture.release()
#화면에 나타난 윈도우창 종료
cv2.destroyAllWindows()

read success!
깨어있는 상태
read success!
깨어있는 상태
read success!
깨어있는 상태
read success!
깨어있는 상태
read success!
깨어있는 상태
read success!
깨어있는 상태
read success!
졸림 상태
read success!
졸림 상태
read success!
졸림 상태
read success!
졸림 상태
read success!
졸림 상태
read success!
졸림 상태
read success!
졸림 상태
read success!
졸림 상태
read success!
졸림 상태
30초 이상 움직임이 감지되지 않았습니다. 졸지마세요!
{'access_token': 'XzLXchmX8vsc1VxOW2l5RRgMqpdoVA-laOvJPQo9c04AAAF-djWEhQ', 'token_type': 'bearer', 'expires_in': 21599}
텍스트 메세지를 성공적으로 보냈습니다.
